In [1]:
from bn256 import *
import random
from copy import copy
import time

In [2]:
def timer(func):
    def _func(*args, **kwargs):
        t = time.clock()
        res = func(*args, **kwargs)
        t = time.clock() - t
        print('{0}: time elapsed {1:.8f} s.' .format(func.__name__, t))
        return res

    return _func

In [3]:
def hamminge_distance(s1, s2):
    l = len(s1)
    assert l == len(s2), "Undefined for sequences of unequal length."
    return sum(el1 != el2 for el1, el2 in zip(s1, s2)) / l

In [4]:
a = (-1, 1)
def gen_vector(size):
    return tuple(random.choice(a) for i in range(size))

In [5]:
v1, v2 = gen_vector(16), gen_vector(16)

In [6]:
print(v1, v2, sep="\n")

(-1, -1, -1, 1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, -1, 1)
(-1, -1, 1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1)


In [7]:
def inverse_G1(curve):
    inv = copy(curve)
    inv.y = inv.y.additive_inverse()
    return inv

In [8]:
def inverse_G2(twist):
    inv = copy(twist)
    inv.negate()
    return inv

In [9]:
def transfer_vec_to_G1(vec):
    return tuple(inverse_G1(curve_G) if p == -1 else curve_G for p in vec)

In [10]:
def transfer_vec_to_G2(vec):
    return tuple(inverse_G2(twist_G) if p == -1 else twist_G for p in vec)

In [11]:
def gen_keys(size):
    return tuple(random.randrange(2, order) for i in range(size))

In [12]:
v1points = transfer_vec_to_G2(v1)
v2points = transfer_vec_to_G1(v2)

In [13]:
v1points

(((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((21167961636542580255011770066570541300993051739349375019639421053990175267184,64746500191241794695844075326670126197795977525365406531717464316923369116492), (44333636345587826776585919623241361010268679977813729575614822660813824120661,47221932139242163798143780279757230461301747781284361464062811170020202647234)),
 ((2116796163654258025501177006657054

In [14]:
v2points

((1, 2),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 65000549695646603732796438742359905742825358107623003571877145026864184071781),
 (1, 2))

In [15]:
h1, h2 = random.randrange(2, order), random.randrange(2, order)
H1, H2 = curve_G.scalar_mul(h1), twist_G.scalar_mul(h2)

In [16]:
h1, h2

(59533400870022841113655934254440531010849439750355421658962753552272763733851,
 50960790229496102605422383493962307210676186302272564129380952304755311666809)

In [17]:
@timer
def master_key_generation():
    s, t = gen_keys(16), gen_keys(16)
    u, v = gen_keys(18), gen_keys(18)
    gen1_h = tuple((curve_G.scalar_mul(si), H1.scalar_mul(ti)) for si, ti in zip(s, t))
    gen2_h = tuple((twist_G.scalar_mul(ui), H2.scalar_mul(vi)) for ui, vi in zip(u, v))
    return gen1_h, s, t, gen2_h, u, v

In [18]:
gen1_h, s, t, gen2_h, u, v = master_key_generation()

master_key_generation: time elapsed 13.56288081 s.


In [19]:
gen1_h, s, t, gen2_h, u, v

((((...))

In [20]:
@timer
def decryption_key_generation(gen2_h, s, t, v1points):
    n = len(v1points)
    
    r0 = random.randrange(2, order)
    r1 = random.randrange(2, order)
    
    gen2_h0 = tuple(point_add(g.scalar_mul(r0), h.scalar_mul(r1)) for g, h in gen2_h)
    
    reg_template = [twist_G.scalar_mul(r0), H2.scalar_mul(r1)]
    
    d03 = v1points[0].scalar_mul(s[0])
    d04 = v1points[0].scalar_mul(t[0])
    for i in range(1, n):
        d03 = point_add(d03, v1points[i].scalar_mul(s[i]))
        d04 = point_add(d04, v1points[i].scalar_mul(t[i]))
    reg_template.append(point_add(inverse_G2(d03), gen2_h0[0]))
    reg_template.append(point_add(inverse_G2(d04), gen2_h0[1]))

    for i in range(n):
        reg_template.append(point_add(v1points[i], gen2_h0[i + 2]))
    
    return tuple(reg_template)

In [21]:
DK = decryption_key_generation(gen2_h, s, t, v1points)

decryption_key_generation: time elapsed 22.40061173 s.


In [22]:
len(DK)

20

In [23]:
DK

(((56778757113100796022805227780968748289999938886854220848900767282156181485479,35759252536065400748365466309485552777764822993794696776973856261628053400206), (36749360499845237349732165807913881078784675259743604488299712529034311977094,49102935726236333666230188665680466414292084813621218731379463535847122337832)),
 ((38130043004915943707115267545324353395207980322522061232180480978916137482299,29424515589348427272917623066690562275175754277719966727241512044629377806199), (45697543250068231747618986504175979308859299007271906339715162615752188940845,63456695766803957786421199710245620838827427511768530928164233996260963671059)),
 ((58785466597611775881829980469181071525230955422077404017344829057711428378299,5245864078201801474732919789253458174106833951228427651105245117624117054538), (21693114568282601661822706951670291187735330751858190967478543568528927055194,30285687765812963944178540756838621889395423155465509286337980916567224014321)),
 ((60141067179581749082523933760222211

In [24]:
@timer
def encryption_authentication(gen1_h, u, v, v2points):
    n = len(v2points)
    
    r2 = random.randrange(2, order)
    r3 = random.randrange(2, order)
    
    gen1_h0 = tuple(point_add(g.scalar_mul(r2), h.scalar_mul(r3)) for g, h in gen1_h)
    
    auth_template = [None, None, curve_G.scalar_mul(r2), H1.scalar_mul(r3)]
    
    c01 = point_add(auth_template[2].scalar_mul(u[0]), auth_template[3].scalar_mul(u[1]))
    c02 = point_add(auth_template[2].scalar_mul(v[0]), auth_template[3].scalar_mul(v[1]))
    for i, (ci, h) in enumerate(zip(v2points, gen1_h0)):
        a = point_add(ci, h)
        auth_template.append(a)
        c01 = point_add(c01, a.scalar_mul(u[i + 2]))
        c02 = point_add(c02, a.scalar_mul(v[i + 2]))
    
    auth_template[0] = inverse_G1(c01)
    auth_template[1] = inverse_G1(c02)
    
    return tuple(auth_template)

In [25]:
CT = encryption_authentication(gen1_h, u, v, v2points)

encryption_authentication: time elapsed 5.88708040 s.


In [26]:
len(CT)

20

In [27]:
CT

((43437693989343872272505098610488802795346699526178330511867221254575747454459, 59216007630861481691192135496398226570267503274459947367303175953657292898203),
 (45296106936017325755338728327923872697366545020130226225080570310738924495030, 29631194138416256925069089006179662354601969142987090324904470422031326082710),
 (8928832378956692932722163851410251047501968209352783785127535710632879994545, 41930247139014251755299620277211292158777293021335493621613966757622841941640),
 (38953777953364702611483528437447155357458361950513143490600675299144760468949, 54260988645711147534076746942862530299990083242145969257913352608294674629188),
 (4497316408341407277779023282046323057336545116770225419994510504979382977027, 13980472553282855224317471817130597513212410266284720342146968297030644218001),
 (20410118726827339373422438752320919823986369389028600001894806953451258486100, 50013658213870562406319465714712444510712066631220143833365904750654121778175),
 (1402878279230182010061450742233165

In [28]:
gt = optimal_ate(twist_G, curve_G)
logarithmic_table = [gt.exp(i) for i in range(16, -1, -2)]
gtm = optimal_ate(twist_G, inverse_G1(curve_G))
logarithmic_table.extend(gtm.exp(i) for i in range(2, 17, 2))

In [29]:
for i, el in enumerate(logarithmic_table):
    print(i, ": ", el)

0 :  (((15429308596100125345042025182838740585933899260990097211943232471185708209892,21036769640615344552925704451163498369215862996620789540674973406371475898801),(34643419400173764267828846220737244479195263121357284576095582127626220190690,39217953906512156784965537049391267021797863115575109425638059426537342418239),(29362508885087302295248031767237294697462853748363034881398247042502825752042,33065606296478674290051087119664030123566328954069047137872363079287669349803)),((44150688783494038190860128993786071429009665932812059212008957591291015396555,25855485444396824150330723346354416914393823131049522641950650987664836921571),(14355433543029713763013415741005074744216778912056694539583678910055649667776,46280493583452157638432486289407345007584489845082308921018418732775447515998),(35438949274284069241468591891906709017152328378869050685915198888251448805767,45559280046590109824720435394540600218774466880098159450808971756061223256757)))
1 :  (((597783055027885551703832584052683

In [30]:
@timer
def decryption_authentication(DK, CT):
    d = optimal_ate(DK[0], CT[0])
    for i in range(1, 20):
        d = d.mul(optimal_ate(DK[i], CT[i]))
    return logarithmic_table.index(d)

In [31]:
a = decryption_authentication(DK, CT)

decryption_authentication: time elapsed 17.03162537 s.


In [32]:
a

9

In [33]:
a / 16

0.5625

In [34]:
hamminge_distance(v1, v2)

0.5625